In [1]:
import pdb
import json
import os
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from transformers import BertForMaskedLM, BertTokenizerFast
import torch
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import BertModel, BertConfig
from transformers import BertTokenizer, BertModel, BertConfig
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# import sys
# print(sys.executable)
# ! /anaconda/envs/azureml_py310_sdkv2/bin/python -m pip install torch

def generate_corpus_function(text_data_path):
   
    corpus_data = {}
    
    try:
        with open(text_data_path, 'r', encoding='utf-8') as file:
            for i, line in enumerate(file.readlines()):
                corpus_data[f'doc_{i}'] = line.strip()  # Creating a dictionary entry for each line
    except FileNotFoundError:
        print("Text data file not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return corpus_data

if __name__ == "__main__":
    
    
    TRAIN_DATASET_FPATH = '/home/azureuser/cloudfiles/code/Users/hb.suh/OUR_BERT/data/train_dataset.json'
    VAL_DATASET_FPATH = '/home/azureuser/cloudfiles/code/Users/hb.suh/OUR_BERT/data/val_dataset.json'

    batch_size=16

    model_name = '/home/azureuser/cloudfiles/code/Users/hb.suh/OUR_BERT/MLM_trained_model/kcbert-base_CyclicLRtriangular-2024-05-01_11-37-24'
    #klue-roberta-large_CyclicLRtriangular-2024-04-26_08-42-52'


    transformer_model = models.Transformer(model_name)

    # Pooling layer 추가
    pooling_model = models.Pooling(transformer_model.get_word_embedding_dimension(),
                                pooling_mode_mean_tokens=True,
                                pooling_mode_cls_token=False,
                                pooling_mode_max_tokens=False)

    # SentenceTransformer에 word_embedding_model과 pooling_model을 추가하여 모델 구성
    model = SentenceTransformer(modules=[transformer_model, pooling_model])


    

    # Define paths to your dataset
    OUTPUT_DIR = '/home/azureuser/cloudfiles/code/Users/hb.suh/OUR_BERT/EXPERIMENT3_DOMAINKCBERT_generated_QAdata/kcbert-base_CyclicLRtriangular-2024-04-30'
    DATA_DIR = '/home/azureuser/cloudfiles/code/Users/hb.suh/OUR_BERT/generated_QAdata'
    TRAIN_QUERIES_FPATH = os.path.join(DATA_DIR, 'train_queries.json')
    TRAIN_DOCS_FPATH = os.path.join(DATA_DIR, 'train_relevant_docs.json')
    VAL_QUERIES_FPATH = os.path.join(DATA_DIR, 'val_queries.json')
    VAL_DOCS_FPATH = os.path.join(DATA_DIR, 'val_relevant_docs.json')

    # Load datasets
    with open(TRAIN_DATASET_FPATH, 'r+') as f:
        train_dataset = json.load(f)
    with open(VAL_DATASET_FPATH, 'r+') as f:
        val_dataset = json.load(f)
        
    with open(TRAIN_QUERIES_FPATH, 'r') as f:
        train_queries = json.load(f)
    with open(TRAIN_DOCS_FPATH, 'r') as f:
        train_relevant_docs = json.load(f)
    with open(VAL_QUERIES_FPATH, 'r') as f:
        val_queries = json.load(f)
    with open(VAL_DOCS_FPATH, 'r') as f:
        val_relevant_docs = json.load(f)

    train_corpus = train_dataset['corpus'] # dic
    train_queries = train_dataset['queries']
    train_relevant_docs = train_dataset['relevant_docs']

    # 쿼리 ID에 따라 관련 문서 텍스트를 추출
    examples = []
    for query_id, query in train_queries.items():
        if query_id in train_relevant_docs:
            node_ids = train_relevant_docs[query_id]  # 문서 번호 리스트
            for node_id in node_ids:
                # train_corpus의 키는 UUID, train_relevant_docs의 값은 문서 번호 리스트
                # 문서 번호(node_id)를 사용하여 해당하는 UUID를 찾아야 함
                uuid_key = None
                for key in train_corpus.keys():
                    # UUID 키에 node_id가 포함되어 있는지 체크 (가정: UUID 내에 문서 번호가 포함되어 있다고 가정)
                    if str(node_id) in key:
                        uuid_key = key
                        break

                if uuid_key:
                    text = train_corpus[uuid_key]  # UUID를 이용해 텍스트 추출
                    example = InputExample(texts=[query, text], label=1)
                    examples.append(example)
                else:
                    print(f"Node ID {node_id} not found in corpus for Query ID {query_id}.")
        else:
            print(f"Query ID {query_id} not found in train_relevant_docs.")

/anaconda/envs/llm-rag-embeddings/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertModel were not initialized from the model checkpoint at /home/azureuser/cloudfiles/code/Users/hb.suh/OUR_BERT/MLM_trained_model/kcbert-base_CyclicLRtriangular-2024-05-01_11-37-24 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
!nvidia-smi

Wed May  1 12:35:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           Off | 00000001:00:00.0 Off |                  Off |
| N/A   29C    P0              37W / 250W |   2522MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
# ! kill -9 6245

In [3]:
def custom_collate_fn(batch):
    texts = [example.texts for example in batch]
    labels = [example.label for example in batch]
    return texts, labels  

train_loader = DataLoader(
    examples, batch_size, collate_fn=custom_collate_fn
)
    
# Define the loss function
loss = losses.MultipleNegativesRankingLoss(model)


######## Define evaluator ##########

val_corpus = val_dataset['corpus']
val_queries = val_dataset['queries']
val_relevant_docs = val_dataset['relevant_docs']


evaluator = InformationRetrievalEvaluator(val_queries, val_corpus, val_relevant_docs)

# Training
EPOCHS = 12
warmup_steps = int(len(train_loader) * EPOCHS * 0.1)  # Adjust warmup steps as needed

model.fit(
    train_objectives=[(train_loader, loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    evaluator=evaluator,
    evaluation_steps=50,
    output_path=os.path.join(OUTPUT_DIR, 'exp_finetune'),  # Save the fine-tuned model
    show_progress_bar=True
)

Epoch: 100%|██████████| 12/12 [1:17:39<00:00, 388.29s/it]


In [4]:
# !pip show torch torchvision torchaudio
# # 기존 토치 : 1.12.0, 토치 비전 : 0.9.1, torchaudio : 2.2.2

Name: torch
Version: 1.12.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: typing-extensions
Required-by: torchvision, torchaudio, sentence-transformers, pytorch-transformers, fastai, azureml-automl-dnn-nlp
---
Name: torchvision
Version: 0.9.1
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: numpy, torch, pillow
Required-by: fastai
---
Name: torchaudio
Version: 2.2.2
Summary: An audio package for PyTorch
Home-page: https://github.com/pytorch/audio
Author: Soumith Chintala, David Pollack, Sean Naren, Peter Goldsborough, Moto Hira, Caroline Chen, Jeff Hwang, Zhaohen

In [5]:
# !pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2

     |████████████████████████████████| 619.9 MB 11 kB/s s eta 0:00:01
     |████████████████████████████████| 33.8 MB 69.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 44.4 MB/s eta 0:00:01
     |████████████████████████████████| 177.1 MB 76 kB/s s eta 0:00:01
     |████████████████████████████████| 54.6 MB 65.7 MB/s eta 0:00:01
     |████████████████████████████████| 63.2 MB 8.3 kB/s  eta 0:00:01
     |████████████████████████████████| 557.1 MB 5.1 kB/s  eta 0:00:01
     |████████████████████████████████| 168.4 MB 74 kB/s s eta 0:00:01
     |████████████████████████████████| 98 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 317.1 MB 17 kB/s s eta 0:00:01
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
     |████████████████████████████████| 849 kB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 11.8 MB 70.0 MB/s eta 0:00:01
     |████████████████████████████████| 173.2 MB 28 kB/s s eta 0:00:01
     |██████████████████████████████

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

